In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline

PATH = "/kaggle/input/tsp-cv"
PATH_TRAIN = os.path.join(PATH, "train.csv")
PATH_TEST = os.path.join(PATH , 'test.csv')

df_train = pd.read_csv(PATH_TRAIN)
df_test = pd.read_csv(PATH_TEST)

In [ ]:
TRAIN_PCT = 0.78
TRAIN_CUT = int(len(df_train) * TRAIN_PCT)

df_train_cut = df_train[0:TRAIN_CUT]
df_validate_cut = df_train[TRAIN_CUT:]

print(f"Training size: {len(df_train_cut)}")
print(f"Validate size: {len(df_validate_cut)}")
display(df_train_cut)
display(df_validate_cut)

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

# WIDTH = 299
# HEIGHT = 299
WIDTH = 299
HEIGHT = 299

training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  #vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=32, # Keeping the training batch size small USUALLY increases performance
        class_mode='raw')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=256, # Make the validation batch size as large as you have memory for
        class_mode='raw')


test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=PATH,
        x_col="filename",
    
        batch_size=1,
        shuffle=False,
        target_size=(HEIGHT, WIDTH),
        class_mode=None)

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
#https://www.kaggle.com/code/abnera/transfer-learning-keras-xception-cnn/script
input_tensor = Input(shape=(HEIGHT, WIDTH, 3))

base_model = InceptionV3(
    include_top=False, weights='imagenet', input_tensor=input_tensor,
    input_shape=None)

#base_model.summary()

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
#https://vigneshgig.medium.com/xception-neural-network-transfer-learning-and-data-processing-using-ai-c3e7a4ea7bf2
#https://www.kaggle.com/code/abnera/transfer-learning-keras-xception-cnn/script
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(2048,activation='relu')(x) 
x=Dense(2048,activation='relu')(x) 
model=Model(inputs=base_model.input,outputs=Dense(1)(x))
# predictions = Dense(2, activation='softmax')(x)
# model = Model(base_model.input, predictions)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow import keras
# Important, calculate a valid step size for the validation dataset
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
base_model.trainable = True
model.compile(loss = 'mean_squared_error', optimizer=keras.optimizers.Adam(1e-5), metrics=[RootMeanSquaredError(name="rmse")])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto',
        restore_best_weights=True)

history = model.fit(train_generator, epochs=120, steps_per_epoch=50, 
                    validation_data = val_generator, callbacks=[monitor],
                    verbose = 1, validation_steps=STEP_SIZE_VALID)

In [ ]:
test_generator.reset()
pred = model.predict(test_generator,steps=len(df_test))

df_submit = pd.DataFrame({'id':df_test['id'],'distance':pred.flatten()})
df_submit.to_csv("submit.csv",index=False)